<center>
    <h3> Time Dependent Density Functional Perturbation Theory with QEpy</h3>
    <h3>Tutorial</h3>
</center>
<table>
  <tr>
      <td><p><h4>Rutgers University-Newark</h4></p><p>Dr Xuecheng Shao</p><p>Dr Kaili Jiang (Microsoft)</p><p>Dr. Xin Chen</p><p>Jessica Martinez</p><p>Valeria Rios </p><p>Alina Umerbekova</p><p>Nicholas Viot</p></td>
          <td><img src="../figures/logos/logo.jpg" width=800 /></td>
  </tr>
</table>

Funding: NSF, DOE, MolSSI, ACS-PRF, Rutgers

## Goal
#### Obtain the Absorption Spectra of Methane implementing turbo davidson
To run this tutorial we need to import QEpy and some of its modules

In [ ]:
import qepy
from qepy.driver import Driver
from qepy.io import QEInput

## Generate the QE options to run a scf calculation

In [ ]:
scf_options = {
    '&control': {'calculation': "'scf'",
                 'prefix': "'CH4'",
                 'restart_mode': "'from_scratch'",
                 'outdir': "'tmp'",
                 'pseudo_dir' : "'./'",
                },
    '&system': {'celldm(1)': 20,
                'ecutwfc': 25,
                'ibrav': 1,
                'nat': 5,
                'nbnd': 20,
                'ntyp': 2},
    '&electrons': {'conv_thr': 1.0E-10},
    'atomic_positions {angstrom}': ['C   0.000000000   0.000000000   0.000000000',
                                    'H   0.642814093   0.642814093   0.642814093',
                                    'H  -0.642814093  -0.642814093   0.642814093',
                                    'H   0.642814093  -0.642814093  -0.642814093',
                                    'H  -0.642814093   0.642814093  -0.642814093'],
    'atomic_species': ['H  1.008  H.pz-vbc.UPF', 'C 12.011  C.pz-vbc.UPF'],
    'k_points {gamma}': [],
}

## Generate the QE options for the TDDFPT calculation

In [ ]:
tddfpt_options = {
    '&lr_input': {'outdir': "'tmp'",
                  'prefix': "'CH4'",
                  'restart': False,
                 },
    '&lr_dav': {'num_basis_max': 80,
                'num_eign': 10,
                'num_init': 20,
                'p_nbnd_occ': 4,
                'p_nbnd_virt': 15,
                'reference': 0.5,
                'residue_conv_thr': 1.0E-4,
                'start': 0.0,
                'finish': 1.5,
                'broadening': 0.005,
                'step': 2.0E-4 ,
               },
}

 Write the options to input files for comparison with traditional command way

In [ ]:
scf_in = 'CH4.scf.in'
tddfpt_in= 'CH4.tddfpt.in'
QEInput().write_qe_input(scf_in, qe_options=scf_options, prog='pw')
QEInput().write_qe_input(tddfpt_in, qe_options=tddfpt_options, prog='davidson')

In [ ]:
!cat CH4.scf.in

Download the PP with DFTpy

In [ ]:
additional_files = {
    'H.pz-vbc.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/H.pz-vbc.UPF',
    'C.pz-vbc.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/C.pz-vbc.UPF',
}
from dftpy.formats import download_files
download_files(additional_files)

Initialize the Driver class with the scf QE options

In [ ]:
driver = Driver(scf_in, task='scf', logfile='tmp.scf.out')

Run a scf calculation

In [ ]:
driver.scf()

We need to save the wave function for the TDDFT calculation

In [ ]:
driver.save()

Initialize the Driver class with the tddfpt QE options to calculate the absorption spectra with truboDavidson

In [ ]:
driver = Driver(tddfpt_in, task='tddfpt_davidson', logfile='tmp.tddfpt.out')

In [ ]:
driver.scf()

In [ ]:
driver.stop()

# Now we can plot the absorption spectra 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.loadtxt('CH4.plot.dat')

In [ ]:
plt.plot(data[:,0], data[:,1])
plt.xlabel(r'$\omega$ (eV)')
plt.ylabel(r'Intensity (arb. units)')

# Challenge

<b>Compute the Absorption spectra of Ethylene C2H4</b>

Important aspect

Create another folder

Hint use ASE to build the molecule
<b>get the scaled positions</b> of the molecule to build the scf_options